<a href="https://colab.research.google.com/github/Pankotaiimre2001/gitt/blob/master/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent

# LLM alapú Ágens készítése a MedQuad adatbázishoz.

A MedQuad (Medical Question Answering Dataset) egy angol nyelvű orvosi adatbázis, amelyet elsősorban mesterséges intelligencia és természetes nyelvi feldolgozás (NLP) kutatásokhoz fejlesztettek ki. Ez az adatbázis kérdés-válasz párokat tartalmaz, amelyek orvosi témákra fókuszálnak, és célja, hogy segítse az automatizált rendszereket abban, hogy pontos és megbízható válaszokat adjanak egészségügyi kérdésekre. Az adatbázis tartalma általában orvosi szakirodalomból, klinikai információkból és hiteles forrásokból származik, például a National Institutes of Health (NIH) vagy más elismert egészségügyi szervezetek publikációiból.

## Függőségek telepítése

In [4]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

## Csomagok importálása

In [21]:
import os
import kagglehub
import pandas as pd

from tqdm import tqdm

from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.memory import ChatMemoryBuffer



from matplotlib import pyplot as plt

In [7]:
# Download latest version
path = kagglehub.dataset_download(handle="googleai/dataset-metadata-for-cord19")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dataset-metadata-for-cord19


In [8]:
os.listdir(path)

['CORD19 datasets - Sheet 1.csv']

In [9]:
filename_with_path = path + "/" + os.listdir(path)[0]
filename_with_path

'/kaggle/input/dataset-metadata-for-cord19/CORD19 datasets - Sheet 1.csv'

In [10]:
df_meta_cord19 = pd.read_csv(filename_with_path)
df_meta_cord19.head()

,cord_uid,paper_url,paper_title,dataset_url,dataset_name,alternate_name,description,author_list,last_updated,license,source_organization,doi,compact_identifier,data_download
0,rmzpiyqj,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,"Nipah virus: epidemiology, pathology, immunobi...",https://data.csiro.au/dap/landingpage?pid=csir...,Nature of exposure drives transmission of Nipa...,NaN,"[""RT-PCR data of comparative viral loads/ tiss...",Bronwyn Clayton; Deborah Middleton; Rachel Ark...,2016,"[{""url"":""https://confluence.csiro.au/display/d...",CSIRO,10.4225/08/56806AAEAD713,NaN,NaN
1,h7g5ecc0,http://europepmc.org/articles/pmc4052367?pdf=r...,Novel approaches and challenges to treatment o...,https://datamed.org/display-item.php?repositor...,Key Role of T cell Defects in Age-Related Vuln...,NaN,"[""In a mouse model of age-related vulnerabilit...",NaN,2019-05-06,NaN,NaN,NaN,NaN,NaN
2,3uvlmww0,https://jvi.asm.org/content/jvi/88/17/10228.fu...,"Verdinexor, a Novel Selective Inhibitor of Nuc...",https://datamed.org/display-item.php?repositor...,MicroRNA Regulation of Human Protease Genes,NaN,the human protease genes required for influenz...,NaN,2011-10-13,NaN,NaN,NaN,NaN,NaN
3,xzps65et,https://doi.org/10.14745/ccdr.v45i04a01,Climate change and infectious diseases: What c...,https://search.datacite.org/works/10.5065/d6sj...,The NA-CORDEX dataset,NaN,"[""The NA-CORDEX data archive contains output f...",Linda Mearns; Seth McGinnis; Daniel Korytina; ...,2017,"[{""url"":""http://na-cordex.org/terms-use""}]",UCAR/NCAR,10.5065/d6sj1jch,NaN,NaN
4,a6p8te8q,https://jvi.asm.org/content/jvi/79/6/3370.full...,Increased Epitope-Specific CD8(+) T Cells Prev...,http://www.immunedata.org/display-item.php?rep...,CMV CD8 T Cells,NaN,"[""We present human T cell responses in multipl...",NaN,2018-09-17,NaN,NaN,NaN,NaN,NaN


In [11]:
df_meta_cord19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16070 entries, 0 to 16069
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cord_uid             16070 non-null  object
 1   paper_url            16070 non-null  object
 2   paper_title          16070 non-null  object
 3   dataset_url          16070 non-null  object
 4   dataset_name         16070 non-null  object
 5   alternate_name       132 non-null    object
 6   description          14126 non-null  object
 7   author_list          6052 non-null   object
 8   last_updated         12548 non-null  object
 9   license              12788 non-null  object
 10  source_organization  12518 non-null  object
 11  doi                  5923 non-null   object
 12  compact_identifier   2080 non-null   object
 13  data_download        5500 non-null   object
dtypes: object(14)
memory usage: 1.7+ MB


In [12]:
df_meta_cord19_filtered = df_meta_cord19[df_meta_cord19['description'].notnull()]
df_meta_cord19_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14126 entries, 0 to 16069
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cord_uid             14126 non-null  object
 1   paper_url            14126 non-null  object
 2   paper_title          14126 non-null  object
 3   dataset_url          14126 non-null  object
 4   dataset_name         14126 non-null  object
 5   alternate_name       132 non-null    object
 6   description          14126 non-null  object
 7   author_list          6052 non-null   object
 8   last_updated         12548 non-null  object
 9   license              12788 non-null  object
 10  source_organization  12516 non-null  object
 11  doi                  5923 non-null   object
 12  compact_identifier   2080 non-null   object
 13  data_download        5500 non-null   object
dtypes: object(14)
memory usage: 1.6+ MB


## Create a vector database using the LlamaIndex function library.

In [13]:
if 'df_meta_cord19_filtered' not in locals() or df_meta_cord19_filtered.empty:
    print("Hiba: A 'df_meta_cord19_filtered' DataFrame nincs definiálva vagy üres.")
    print("Kérlek, töltsd be az adatokat a DataFrame-be a szkript futtatása előtt.")
    # Kilépés, ha a DataFrame nincs rendben
    # exit() # Ezt a sort éles környezetben használd, ha szükséges
else:
    print("A 'df_meta_cord19_filtered' DataFrame sikeresen megtalálva.")

    # Transform the DataFrame into a list of Document objects
    # JAVÍTÁS: 'title' -> 'paper_title', 'sha' -> 'cord_uid' (vagy más releváns egyedi azonosító)
    print("Dokumentumok átalakítása...")
    documents = []
    for index, row in tqdm(df_meta_cord19_filtered.iterrows(), total=len(df_meta_cord19_filtered), desc="Dokumentumok feldolgozása"):
        # Biztonsági ellenőrzés, hogy a 'description' oszlop létezik és nem NaN
        description_text = str(row['description']) if pd.notna(row['description']) else ""
        paper_title_text = str(row['paper_title']) if pd.notna(row['paper_title']) else "Nincs cím"
        cord_uid_text = str(row['cord_uid']) if pd.notna(row['cord_uid']) else f"ismeretlen_uid_{index}"

        documents.append(Document(
            text=description_text,
            metadata={
                'title': paper_title_text,
                'sha': cord_uid_text # Vagy használhatod a 'doi' oszlopot is: row['doi']
            }
        ))
    print(f"{len(documents)} dokumentum sikeresen átalakítva.")

    # Set up the embedding model
    # Győződj meg róla, hogy a sentence-transformers könyvtár telepítve van:
    # pip install sentence-transformers
    print("Beágyazó modell beállítása...")
    try:
        Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
        print("Beágyazó modell sikeresen beállítva.")
    except Exception as e:
        print(f"Hiba a beágyazó modell beállításakor: {e}")
        print("Lehet, hogy nincs internetkapcsolat, vagy a modellnév hibás.")
        # exit() # Éles környezetben

    # Define the storage directory
    storage_dir = "./cord19_metadata_index_corrected" # Kicsit átneveztem, hogy ne írja felül a régit, ha van
    print(f"Tárolási könyvtár: {storage_dir}")

    # Check if the index already exists, if not create it
    if not os.path.exists(storage_dir):
        print("Új index létrehozása...")
        try:
            # Create the VectorStoreIndex from the documents
            index = VectorStoreIndex.from_documents(documents, show_progress=True)
            # Persist the index to disk
            index.storage_context.persist(persist_dir=storage_dir)
            print("Új index sikeresen létrehozva és mentve.")
        except Exception as e:
            print(f"Hiba az index létrehozása vagy mentése közben: {e}")
            # exit() # Éles környezetben
    else:
        print("Meglévő index betöltése...")
        try:
            # Load the index from disk
            storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
            index = load_index_from_storage(storage_context)
            print("Meglévő index sikeresen betöltve.")
        except Exception as e:
            print(f"Hiba a meglévő index betöltése közben: {e}")
            # exit() # Éles környezetben

    if 'index' in locals():
        print("A vektoradatbázis sikeresen létrehozva vagy betöltve.")
        # Itt már használhatod az 'index' objektumot keresésekhez, stb.
        # Példa lekérdezés (ha az index sikeresen létrejött/betöltődött):
        # query_engine = index.as_query_engine()
        # response = query_engine.query("Mi a koronavírus?")
        # print("Példa lekérdezés eredménye:", response)
    else:
        print("Nem sikerült létrehozni vagy betölteni az indexet.")

A 'df_meta_cord19_filtered' DataFrame sikeresen megtalálva.
Dokumentumok átalakítása...


Dokumentumok feldolgozása: 100%|██████████| 14126/14126 [00:01<00:00, 9996.43it/s] 


14126 dokumentum sikeresen átalakítva.
Beágyazó modell beállítása...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1955 [00:00<?, ?it/s]

Új index sikeresen létrehozva és mentve.
A vektoradatbázis sikeresen létrehozva vagy betöltve.


## Create Agent

In [19]:

# Létrehozzuk a nyelvi modellt (LLM), amit az ágens fog használni.
llm = HuggingFaceLLM(
    model_name="colesmcintosh/Llama-3.2-1B-Instruct-Mango",       # Nyelvi modell beállítása
    tokenizer_name="colesmcintosh/Llama-3.2-1B-Instruct-Mango",   # Nyelvi modell tokenizátorának beállítása
    context_window=2048,                                          # Maximum token limit
    max_new_tokens=256,                                           # Válasz maximális hossza
    device_map="cuda:0",                                          # GPU használata,
    generate_kwargs={"temperature": 0.95, "do_sample": True},     # Ezek a paraméterek befolyásolják a modell válaszainak véletlenszerűségét és kreativitását.
)

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
Settings.llm = llm

In [24]:
# Létrehozzuk a chat motort, ami az ágens párbeszédéért felelős.
chat_engine = index.as_chat_engine(
    llm=llm,
    # Ez a paraméter beállítja, hogy a chat motor a korábban létrehozott vektoradatbázist használja a válaszokhoz.
    chat_mode="context",
    # Ez a paraméter beállítja a chat motor memóriáját. A ChatMemoryBuffer emlékszik a korábbi beszélgetésekre.
    memory=ChatMemoryBuffer.from_defaults(token_limit=32000),
    # Ez a paraméter beállítja a rendszerüzenetet, ami az ágens viselkedését befolyásolja. Ebben az esetben az ágens egy orvosi chatbot, amely a MedQuad adathalmaz alapján válaszol.
    system_prompt=(
        "You are a medical chatbot, able to have normal interactions. You only answre based on MedQuad dataset."
    )
)

In [26]:
# Ez egy végtelen ciklus, ami lehetővé teszi, hogy folyamatosan kérdéseket tegyünk fel az ágensnek.
while True:
  query = input("> ")                                     # Bekérjük a felhasználótól a kérdést.
  if query.lower() == "quit":                             # Ha a felhasználó azt írja, hogy "quit", akkor a ciklus megszakad.
      break

  # Streamelt válasz
  print("Agent: ", end="", flush=True)                    # Kiírjuk, hogy az ágens válaszolni fog.
  response = chat_engine.stream_chat(query)               # Ez a sor elküldi a kérdést a chat motornak, és megkapjuk a választ.
  for token in response.response_gen:                     # Ez a ciklus kiírja a választ részleteiben (tokenekként) streamelve.
      print(token, end="", flush=True)
  print()                                                 # Egy új sort írunk ki a válasz után.

chat_engine.reset()                                       # History törlése

> What are the mean, median, minimum, and maximum values of median_house_value in the dataset?
Agent: Median_house_value = 1428.11
> What is the average housing_median_age?
Agent:  median_house_value = 1428.11
> What are the most common categories in the ocean_proximity column, and what are their proportions?
Agent: Here are the most common categories in ocean_proximity and their proportions:
 ocean_proximity: bathymetry | 0.32 | bathymetry | 0.18 | coastline | 0.20 |  | median_house_value | 1428.11 |
> Are there any missing values in the total_bedrooms column? If so, how many? (This is a common characteristic of this dataset.)
Agent: No
> What is the standard deviation of median_income?
Agent:  median_income = 41,121
> are you sure?
Agent:  median_income = 41,121
> What is the range of longitude and latitude values? Based on this, which part of California do the data likely represent?
Agent: The range of longitude values is -122.7 to 121.3.
Based on the latitude values, the data likel